In [26]:
import requests
import os
from dotenv import load_dotenv
import google.generativeai as genai

In [30]:
load_dotenv()

True

In [64]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
MODEL = "gemini-2.5-flash"

In [65]:
reference_prompt = open('prompt_text.txt', "r").read()

In [ ]:
def ask_questions(user_input):
    system_instruction = (
        "You are an expert creative director and prompt engineer. "
        "Given a short user input for an AI-generated moodboard. "
        "ask 3-5 precise clarifying questions. "
        "The answers should help you later expand the input into a professional-grade prompt."
    )
    model = genai.GenerativeModel(model_name=MODEL, system_instruction=system_instruction)
    response = model.generate_content(user_input)
    return response.text.strip()

In [68]:
def expand_prompt(user_input, answers, reference_format):
    system_instruction = (
        "You are an expert creative strategist and prompt engineer. "
        "Take the original input and the user's answers to clarifying questions, "
        "and expand them into a polished, structured professional prompt for AI image generation."
        "Use the following reference format as a guide:\n" + reference_format
    )
    model = genai.GenerativeModel(model_name=MODEL, system_instruction=system_instruction)
    response = model.generate_content(f"Input: {user_input}\nAnswers: {answers}")
    return response.text.strip()

In [56]:
user_input = "Create a moodboard for a Scandinavian bedroom"
questions = ask_questions(user_input)
print("Questions for user:\n", questions)

Questions for user:
 Excellent! "Scandinavian bedroom" is a great starting point. To make sure the AI generates *exactly* the moodboard you envision, let's refine this a bit.

Please tell me:

1.  What specific **mood or feeling** are you hoping to evoke? (e.g., cozy and "hygge," minimalist and serene, bright and airy, rustic and natural, or a more modern and sleek vibe?)
2.  What is your preferred **color palette**? Are we leaning towards very light neutrals (whites, light greys, creams), or would you like to incorporate subtle pastels, muted earth tones, or perhaps a bolder accent color?
3.  Regarding **materials and textures**, what do you envision as primary? (e.g., light wood, warm wool, crisp linen, sheepskin, ceramics, sleek metal, or a combination?)
4.  How important is **natural light**, and what kind of artificial lighting (e.g., soft ambient glow, sculptural lamps, minimalist fixtures) should be present in the moodboard?
5.  Are there any **specific elements or details** you

In [69]:
answers = """
1. Minimalist furniture, clean lines.
2. Color palette should be related to grey.
3. Include textures like wood, linen, and wool.
4. Natural light is important.
5. Include cactus if you can
"""

expanded = expand_prompt(user_input, answers, reference_format=reference_prompt)
print("\nFinal Expanded Prompt:\n", expanded)



Final Expanded Prompt:
 Act as an expert Art Director and Brand Strategist. Your task is to generate a professional, cohesive mood board that visually captures the essence of a serene and inviting Scandinavian bedroom concept.

The mood board must be arranged in a clean 4x3 grid and contain 9-16 distinct visual elements. The overall aesthetic is minimalist, warm, cozy, and deeply connected to nature, emphasizing a calm and uncluttered environment.

Content Distribution & Elements:
The board must feature a carefully curated content mix:

*   **Hero Images (~40%):** Primary visuals that anchor the theme, such as high-quality photography of bedrooms bathed in abundant natural light, showcasing minimalist furniture with clean lines. Focus on the spaciousness and serene atmosphere of the entire room.
*   **Supporting Visuals (~25%):** Secondary images that reinforce the concept, like close-ups of specific Scandinavian decor items, a perfectly made bed, and a subtle yet elegant cactus plant

In [59]:
url = "https://ai-collab.openai.azure.com/openai/deployments/dall-e-3/images/generations?api-version=2024-02-01"
api_key = os.getenv("AZURE_OPENAI_API_KEY")
headers = {
    "api-key": api_key,
    "Content-Type": "application/json"
}

In [70]:
prompt_val = expanded


In [71]:
data = {
    "prompt": prompt_val,
    "size": "1024x1024",
    "n": 1
}

In [72]:
response = requests.post(url, headers=headers, json=data)
print(response.json())

{'created': 1755315191, 'data': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'prompt_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'profanity': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'revised_prompt': 'Create a Scandinavian-inspired mood board arranged in a clean 4x3 grid. Include 9-16 distinct visual elements to capture the essence of a serene and inviting Scandinavian bedroom concept. The layout features: 1. **Hero Images (~40% of the board)**: High-quality visuals of bedrooms bathed in natural light, showcasing minimalist furniture, spaciousness, and a serene atmosphere. Focus on clean lines and warm tones with an un